In [ ]:
import jieba
# jieba.initialize()  # 手动初始化
# jieba采用延迟加载，import jieba不会立即触发词典的加载。

## 1. 分词

- <code>jieba.cut(line, cut_all=False, HMM=True)</code>
- 精确模式，cut_all=False，默认。精确分词，适合文本分析
- 全局模式，cut_all=True。所有可以成词的都扫描出来，速度快，不能解决歧义
- 搜索引擎模式，cut_for_search。在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
- paddle模式，use_paddle=True。利用PaddlePaddle深度学习框架，训练序列标注（双向GRU）网络模型实现分词。

In [ ]:
line = '出  20  长乐农商  05.28  3580'
seg_1 = jieba.cut(line) # default
print('seg_1:', list(seg_1))  

seg_2 = jieba.lcut(line, HMM=False)
print('seg_2:', seg_2)

seg_3 = jieba.lcut_for_search(line, HMM=False)
print('seg_3:', seg_3)

jieba.enable_paddle()
seg_4 = jieba.lcut(line, use_paddle=True)
print('seg_4:', seg_4)

### 日期切分

- paddle模式下，对日期的切分效果明显好于其它模式

In [ ]:
lines = ['180天内 年内 三个月起 90-180天 超期 足月 三月一日到五月一日',
         '2020.12.18 2020/3/5 2020年3月5日 2020-3-5',
         '12月份 9月份到 1月后 11月1日 1月30号 10/1-12/30到',
         '五个月后 五个月起 5个月起 5个月后 5个月',
         '12个月内 一年以内 一年内 1年内 不限日期 不看日期 不限期 不限时间 不分日期 一年期 12个月',
         '1月份到 1月到 1月31日前到 1月前到 1月31日到 1月31日']

jieba.enable_paddle()
for i, line in enumerate(lines):
    seg = jieba.lcut(line, use_paddle=True)
    print('line'+str(i), seg)

## 2 词典

- 通过自定义词典，包含jieba词库中没有的词
- 用法: <code>jieba.load_userdict(file_name)</code>
- 词频省略时使用自动计算的能保证分出该词的词频，
- 自动计算的词频在使用HMM新词发现功能时可能失效

In [ ]:
jieba.load_userdict('user_dict.txt')
line = '12月份 9月份到 1月后 11月1日 1月30号 10/1-12/30到'
seg_5 = jieba.lcut(line)
print('seg_5:', seg_5)

jieba.suggest_freq('月份到', True)  # 合并或分开，需关掉HMM
seg_6 = jieba.lcut(line, HMM=False)
print('seg_6:', seg_6)
seg_7 = jieba.lcut(line)
print('seg_7:', seg_7)

## 3 关键词抽取
### 3.1基于TF-IDF算法的关键词抽取
- <code>jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())</code>
    - withWeight 是否一并返回关键词权重值
    - allowPOS 仅包括指定词性的词
- 逆向文件频率（IDF)和停止词语料库可以切换成自定义语料库路径

In [ ]:
txt='那些你很冒险的梦，我陪你去疯，折纸飞机碰到雨天终究会坠落，伤人的话我直说，因为你会懂，冒险不冒险你不清楚，折纸飞机也不会回来，做梦的人睡不醒！'
Key=jieba.analyse.extract_tags(txt,topK=3, withWeight=True)
print(Key)

### 3.2 基于TextRank算法的关键词抽取
- <code>jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))</code>
    - allowPOS的默认值与TF-IDF不同

In [ ]:
txt='那些你很冒险的梦，我陪你去疯，折纸飞机碰到雨天终究会坠落，伤人的话我直说，因为你会懂，冒险不冒险你不清楚，折纸飞机也不会回来，做梦的人睡不醒！'
key = jieba.analyse.textrank(txt, topK=3, withWeight=True, allowPOS=('n'))
print(key)

## 4 词性标注
- paddle模式词性和专名类别标签集合如下表，其中词性标签24个，专名类别标签4个

|标签 |含义|标签|含义|标签|含义|标签|含义|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|n|普通名词|f|方位名词|s|处所名词|t|时间|
|nr|人名|ns|地名|nt|机构名|nw|作品名|
|nz|其他专名|v|普通动词|vd|动副词|vn|名动词|
|a|形容词|ad|副形词|an|名形词|d|副词|
|m|数量词|q|量词|r|代词|p|介词|
|c|连词|u|助词|xc|其他虚词|w|标点符号|
|PER|人名|LOC|地名|ORG|机构名|TIME|时间|

In [ ]:
import jieba.posseg as pseg
line = '出 20 长乐农商 05.28 3580 12月31日 半年'
words = pseg.cut(line)
print(list(words))

In [ ]:
jieba.enable_paddle()
words = pseg.cut(line, use_paddle=True)
print(list(words))

## 5 并行分词

- 目标文本按行分隔后，各行文本分配到多个Python进程进行分词，然后归并结果
- 基于Python自带的multiprocessing模块，目前暂不支持Windows
- 用法：<code>jieba.enable_paralled(4)</code>
